In [26]:
import torch
import lightning as L
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import BatchSizeFinder, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import NeptuneLogger
import neptune
import os
import torch.autograd.functional as F


### Loading and PreProcessing the Data


In [27]:
#relevent columes

relevent_features = ['_totl,_time','Vtrue,_ktgs','pitch,__deg','_roll,__deg','hding,_true','__lat,__deg','__lon,__deg','___CG,ftMSL','____X,____m','____Y,____m','____Z,____m','___vX,__m/s','___vY,__m/s','___vZ,__m/s'
                     ,'_elev,stick','ailrn,stick','ruddr,stick','thro1,engin'
]

train_data = pd.read_csv('data/Data4.csv')
train_data = train_data.dropna(axis=1, how='all')
train_data = train_data[relevent_features]
train_data

val_data = pd.read_csv('data/Data3.csv')
val_data = val_data.dropna(axis=1, how='all')
val_data = val_data[relevent_features]
val_data

test_data = pd.read_csv('data/Data2.csv')
test_data = test_data.dropna(axis=1, how='all')
test_data = test_data[relevent_features]
test_data

,"_totl,_time","Vtrue,_ktgs","pitch,__deg","_roll,__deg","hding,_true","__lat,__deg","__lon,__deg","___CG,ftMSL","____X,____m","____Y,____m","____Z,____m","___vX,__m/s","___vY,__m/s","___vZ,__m/s","_elev,stick","ailrn,stick","ruddr,stick","thro1,engin"
0,5.48427,103.49908,0.07651,-0.00486,316.95850,46.56116,-122.69409,8150.02539,23464.83203,2437.26929,-6846.28711,-36.49474,-0.21043,-38.77045,-0.04573,0.00032,-0.00179,0.62949
1,5.60311,103.48605,0.33557,-0.02232,316.97202,46.56120,-122.69415,8149.85596,23460.49609,2437.22852,-6850.89453,-36.49013,-0.44140,-38.76478,-0.06844,-0.00653,-0.00463,0.00000
2,5.71961,103.43369,0.62848,-0.03600,316.98975,46.56124,-122.69421,8149.62158,23456.24609,2437.16772,-6855.40918,-36.47029,-0.57554,-38.74413,-0.05645,-0.00832,-0.00633,0.00000
3,5.80210,103.37489,0.83224,-0.04432,317.00247,46.56127,-122.69424,8149.43262,23453.23828,2437.11768,-6858.60400,-36.44852,-0.62677,-38.72172,-0.04408,-0.00867,-0.00709,0.00000
4,5.89681,103.29092,1.05690,-0.05146,317.01611,46.56130,-122.69429,8149.20410,23449.78711,2437.05688,-6862.26953,-36.41737,-0.65032,-38.68958,-0.03029,-0.00876,-0.00772,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7790,677.01807,102.64950,5.03510,-15.52756,92.21129,46.76450,-122.64340,10524.41309,27253.54688,3081.13818,-29478.54102,52.76324,2.18193,2.32113,0.01481,-0.03281,-0.00584,1.00000
7791,677.11859,102.61120,5.03769,-15.75969,91.90834,46.76450,-122.64333,10525.20117,27258.84961,3081.35693,-29478.32422,52.75458,2.17173,2.05556,0.01505,-0.03179,-0.00577,1.00000
7792,677.21912,102.57314,5.04044,-15.98804,91.60071,46.76449,-122.64326,10525.98730,27264.15039,3081.57471,-29478.13281,52.74484,2.16175,1.78573,0.01529,-0.03097,-0.00570,1.00000
7793,677.31958,102.53516,5.04336,-16.21258,91.28848,46.76449,-122.64320,10526.76953,27269.45117,3081.79126,-29477.96875,52.73388,2.15198,1.51168,0.01552,-0.02800,-0.00564,1.00000


# CREATING A MODEL

In [28]:
class FlightDataset(Dataset):
    def __init__(self, df, seq_len, scaler=None):
        # Keeping only the relevant features
        relevant_features = ['_totl,_time','Vtrue,_ktgs','pitch,__deg','_roll,__deg','hding,_true','__lat,__deg','__lon,__deg','___CG,ftMSL','____X,____m','____Y,____m','____Z,____m','___vX,__m/s','___vY,__m/s','___vZ,__m/s'
                     ,'_elev,stick','ailrn,stick','ruddr,stick','thro1,engin'
]
        
        self.df = df[relevant_features].copy()  # Use copy to avoid modifying the original DataFrame

        # Changing the names of the features for easier use
        self.df.columns = ['time', 'vt', 'pitch', 'roll', 'hding', 'lat', 'lon', 'alt', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'elev_stick', 'ailrn_stick', 'ruddr_stick', 'throttle']
        
        if scaler is None:
            self.scaler = StandardScaler()
            self.df = pd.DataFrame(self.scaler.fit_transform(self.df), columns=self.df.columns)
        else:
            self.scaler = scaler
            self.df = pd.DataFrame(self.scaler.transform(self.df), columns=self.df.columns)
        
        # Define sequence length
        self.seq_len = seq_len

    def __len__(self):
        # Making the length shorter by seq_len so we always have a next_state
        return len(self.df) - 1

    def __getitem__(self, idx):
        if idx < self.seq_len:
            padding = np.zeros((self.seq_len - idx, len(self.df.columns)))
            current_state = np.vstack((padding, self.df.iloc[:idx].values.astype(float)))
        else:
            current_state = self.df.iloc[idx - self.seq_len:idx].values.astype(float)
            
        # Next state is the state at idx + seq_len
        next_state = self.df.iloc[idx + 1][['vt', 'pitch', 'roll', 'hding', 'lat', 'lon', 'alt', 'x', 'y', 'z', 'vx', 'vy', 'vz']].values.astype(float)
        full_next_state = self.df.iloc[idx + 1][['time', 'vt', 'pitch', 'roll', 'hding', 'lat', 'lon', 'alt', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'elev_stick', 'ailrn_stick', 'ruddr_stick', 'throttle']].values.astype(float)
        return torch.tensor(current_state, dtype=torch.float32), torch.tensor(next_state, dtype=torch.float32), torch.tensor(full_next_state, dtype=torch.float32)
    
    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

In [29]:
class FlightLSTM(pl.LightningModule):
    def __init__(self,input_dim,hidden_dim,num_layers,output_dim,lr=0.001):
        super(FlightLSTM,self).__init__()
        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.lr = lr
        self.criterion = nn.MSELoss(reduction='none')

        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []
        
    def forward(self,x):
        out,_ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out
    
    def compute_metrics(self, y_true, y_pred):
        mse = np.mean((y_true - y_pred) ** 2, axis=0)
        mae = np.mean(np.abs(y_true - y_pred), axis=0)
        return mse, mae
        

    def physics_loss(self, x, y, y_pred):
        """
        Calculate the physics-based loss for the given predictions.

        Args:
        - x: Input tensor (shape: [batch_size, seq_len, num_features]).
        - y: Target tensor (shape: [batch_size, num_output_features]).
        - y_pred: Predicted tensor (shape: [batch_size, num_output_features]).

        Returns:
        - loss: Physics-based loss (scalar).
        """
        m = 1127.177  # Total mass of the Cessna Skyhawk in kg

        def compute_velocities(time):
            out, _ = self.lstm(x)
            out = self.fc(out[:, -1, :])
            vx_pred = out[:, 6]
            vy_pred = out[:, 7]
            vz_pred = out[:, 8]
            return vx_pred, vy_pred, vz_pred
        
        def compute_actual_velocities(time):
            vx_actual = y[:, 6]
            vy_actual = y[:, 7]
            vz_actual = y[:, 8]
            return vx_actual, vy_actual, vz_actual

        # Extract the relevant features from y_pred
        vx_pred = y_pred[:, 6]
        vy_pred = y_pred[:, 7]
        vz_pred = y_pred[:, 8]
        pitch_pred = y_pred[:, 3]
        roll_pred = y_pred[:, 4]
        hding_pred = y_pred[:, 5]
        # Extract time feature
        time = x[:, :, 0]  # Assuming index 0 corresponds to 'time'
        time.requires_grad_(True)
        # Calculate derivatives with respect to time for the predicted velocities
        vx_pred, vy_pred, vz_pred = compute_velocities(time)
        jacobians = F.jacobian(compute_velocities, time, create_graph=True, strict=False, vectorize=True)
        dvx_dt_pred, dvy_dt_pred, dvz_dt_pred = jacobians

        vx_pred = vx_pred.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]
        vy_pred = vy_pred.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]
        vz_pred = vz_pred.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]

        pitch_pred = pitch_pred.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]
        roll_pred = roll_pred.unsqueeze(1).unsqueeze(2)    # Shape: [64, 1, 1]
        hding_pred = hding_pred.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]

        
        # Calculate predicted forces
        X_pred = m * (dvx_dt_pred + pitch_pred * vz_pred - hding_pred * vy_pred)
        Y_pred = m * (dvy_dt_pred + hding_pred * vx_pred - roll_pred * vz_pred)
        Z_pred = m * (dvz_dt_pred + roll_pred * vy_pred - pitch_pred * vx_pred)

        # Extract the relevant features from y
        pitch_actual = y[:, 3]  # Index 3 corresponds to 'pitch'
        roll_actual = y[:, 4]  # Index 4 corresponds to 'roll'
        hding_actual = y[:, 5]  # Index 5 corresponds to 'hding'
        vx_actual = y[:, 6]
        vy_actual = y[:, 7]
        vz_actual = y[:, 8]

        # Calculate derivatives with respect to time for the actual velocities
        vx_actual, vy_actual, vz_actual = compute_actual_velocities(time)
        jacobians_actual = F.jacobian(compute_actual_velocities, time, create_graph=True, strict=False, vectorize=True)
        dvx_dt_actual, dvy_dt_actual, dvz_dt_actual = jacobians_actual

        vx_actual = vx_actual.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]
        vy_actual = vy_actual.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]
        vz_actual = vz_actual.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]

        pitch_actual = pitch_actual.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]
        roll_actual = roll_actual.unsqueeze(1).unsqueeze(2)    # Shape: [64, 1, 1]
        hding_actual = hding_actual.unsqueeze(1).unsqueeze(2)  # Shape: [64, 1, 1]

        # Calculate actual forces
        X_actual = m * (dvx_dt_actual + pitch_actual * vz_actual - hding_actual * vy_actual)
        Y_actual = m * (dvy_dt_actual + hding_actual * vx_actual - roll_actual * vz_actual)
        Z_actual = m * (dvz_dt_actual + roll_actual * vy_actual - pitch_actual * vx_actual)

        # Compute physics-based loss
        loss_X = torch.mean((X_pred - X_actual) ** 2)
        loss_Y = torch.mean((Y_pred - Y_actual) ** 2)
        loss_Z = torch.mean((Z_pred - Z_actual) ** 2)

        physics_loss = loss_X + loss_Y + loss_Z
        return physics_loss

        
    
    def training_step(self,batch,batch_idx):
        x, y ,_ = batch
        output = self(x)
        mse_loss = self.criterion(output, y).mean()
        phys_loss = self.physics_loss(x, y, output)
        loss = mse_loss + phys_loss
        self.log("train/batch/loss", loss, prog_bar=False)

        y_true = y.cpu().detach().numpy()
        y_pred = output.cpu().detach().numpy()

        mse, mae = self.compute_metrics(y_true, y_pred)

        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"train/batch/mse_{feature}", mse[i], prog_bar=False)
            self.log(f"train/batch/mae_{feature}", mae[i], prog_bar=False)

        self.training_step_outputs.append({"loss": loss, "mse": mse, "mae": mae, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "mse": mse, "mae": mae}
    
    def on_train_epoch_end(self):
        loss = np.array([output["loss"].detach().cpu().numpy() for output in self.training_step_outputs])
        mse = np.vstack([output["mse"] for output in self.training_step_outputs])
        mae = np.vstack([output["mae"] for output in self.training_step_outputs])

        y_true_all = np.vstack([output["y_true"] for output in self.training_step_outputs])
        y_pred_all = np.vstack([output["y_pred"] for output in self.training_step_outputs])

        mse_all = np.mean(mse, axis=0)
        mae_all = np.mean(mae, axis=0)
        r2_all = r2_score(y_true_all, y_pred_all)

        self.log("train/epoch/loss", loss.mean())  # Log training epoch loss
        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"train/epoch/mse_{feature}", mse_all[i])  # Log training epoch MSE for each feature
            self.log(f"train/epoch/mae_{feature}", mae_all[i])  # Log training epoch MAE for each feature
        self.log("train/epoch/r2", r2_all)  # Log training epoch R²


        print("Training Epoch End: Loss:", loss.mean(), "MSE:", mse_all, "MAE:", mae_all, "R²:", r2_all)  # Debugging print
        
        self.training_step_outputs.clear()

    def validation_step(self, batch, batch_idx):
        x, y, y_full = batch
        output = self.rolling_prediction(x[0], y_full)
        mse_loss = self.criterion(output, y).mean()
        phys_loss = self.physics_loss(x, y, output)
        loss = mse_loss + phys_loss

        y_true = y.cpu().detach().numpy()
        y_pred = output.cpu().detach().numpy()

        mse, mae = self.compute_metrics(y_true, y_pred)

        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"val/batch/mse_{feature}", mse[i], prog_bar=False)
            self.log(f"val/batch/mae_{feature}", mae[i], prog_bar=False)

        self.validation_step_outputs.append({"loss": loss, "mse": mse, "mae": mae, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "mse": mse, "mae": mae}
    
    def on_validation_epoch_end(self):
        loss = np.array([output["loss"].detach().cpu().numpy() for output in self.validation_step_outputs])
        mse = np.vstack([output["mse"] for output in self.validation_step_outputs])
        mae = np.vstack([output["mae"] for output in self.validation_step_outputs])

        y_true_all = np.vstack([output["y_true"] for output in self.validation_step_outputs])
        y_pred_all = np.vstack([output["y_pred"] for output in self.validation_step_outputs])

        mse_all = np.mean(mse, axis=0)
        mae_all = np.mean(mae, axis=0)
        r2_all = r2_score(y_true_all, y_pred_all)

        self.log("val/epoch/loss", loss.mean())  # Log validation epoch loss
        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"val/epoch/mse_{feature}", mse_all[i])  # Log validation epoch MSE for each feature
            self.log(f"val/epoch/mae_{feature}", mae_all[i])  # Log validation epoch MAE for each feature
        self.log("val/epoch/r2", r2_all)  # Log validation epoch R²

        print("Validation Epoch End: Loss:", loss.mean(), "MSE:", mse_all, "MAE:", mae_all, "R²:", r2_all)  # Debugging print

        self.validation_step_outputs.clear()  # free memory

    def test_step(self, batch, batch_idx):
        x, y, y_full = batch
        output = self.rolling_prediction(x[0], y_full)
        loss = self.criterion(output, y).mean()

        y_true = y.cpu().detach().numpy()
        y_pred = output.cpu().detach().numpy()

        mse, mae = self.compute_metrics(y_true, y_pred)

        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"test/batch/mse_{feature}", mse[i], prog_bar=False)
            self.log(f"test/batch/mae_{feature}", mae[i], prog_bar=False)

        self.test_step_outputs.append({"loss": loss, "mse": mse, "mae": mae, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "mse": mse, "mae": mae}

    def on_test_epoch_end(self):
        loss = np.array([output["loss"].detach().cpu().numpy() for output in self.test_step_outputs])
        mse = np.vstack([output["mse"] for output in self.test_step_outputs])
        mae = np.vstack([output["mae"] for output in self.test_step_outputs])

        y_true_all = np.vstack([output["y_true"] for output in self.test_step_outputs])
        y_pred_all = np.vstack([output["y_pred"] for output in self.test_step_outputs])

        mse_all = np.mean(mse, axis=0)
        mae_all = np.mean(mae, axis=0)
        r2_all = r2_score(y_true_all, y_pred_all)

        self.log("test/epoch/loss", loss.mean())  # Log test epoch loss
        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"test/epoch/mse_{feature}", mse_all[i])  # Log test epoch MSE for each feature
            self.log(f"test/epoch/mae_{feature}", mae_all[i])  # Log test epoch MAE for each feature
        self.log("test/epoch/r2", r2_all)  # Log test epoch R²

        print("Test Epoch End: Loss:", loss.mean(), "MSE:", mse_all, "MAE:", mae_all, "R²:", r2_all)  # Debugging print

        self.test_step_outputs.clear()  # free memory

    def save_predictions_to_csv(self, test_data):
        predictions = []
        actual_data = []
        
        for batch in test_data:
            x, y = batch
            output = self.forward(x)
            predictions.append(output.cpu().detach().numpy())
            actual_data.append(y.cpu().detach().numpy())
            
        predictions = np.vstack(predictions)
        actual_data = np.vstack(actual_data)

        # Use the scaler to inverse transform only the predictions and actuals, ensuring alignment with the scaler's feature order
        num_samples = predictions.shape[0]
        all_features = ['time', 'vt', 'pitch', 'roll', 'hding', 'lat', 'lon', 'alt', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'elev_stick', 'ailrn_stick', 'ruddr_stick', 'throttle']
        relevant_indices = [all_features.index(feature) for feature in ['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']]
        
        # Create a zero matrix with the same shape as the scaler's expected input
        padded_predictions = np.zeros((num_samples, len(all_features)))
        padded_actual_data = np.zeros((num_samples, len(all_features)))

        # Place the predictions and actual data in the correct positions
        padded_predictions[:, relevant_indices] = predictions
        padded_actual_data[:, relevant_indices] = actual_data

        # Inverse transform
        scaler = test_data.dataset.scaler
        unnormalized_predictions = scaler.inverse_transform(padded_predictions)[:, relevant_indices]
        unnormalized_actual_data = scaler.inverse_transform(padded_actual_data)[:, relevant_indices]

        # Create dataframes for the unnormalized predictions and actual values
        columns = ['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']
        predictions_df = pd.DataFrame(unnormalized_predictions, columns=[f'predicted_{col}' for col in columns])
        actual_data_df = pd.DataFrame(unnormalized_actual_data, columns=[f'actual_{col}' for col in columns])
        
        # Combine the dataframes such that each predicted value is next to its corresponding actual value
        combined_df = pd.DataFrame()
        for col in columns:
            combined_df[f'predicted_{col}'] = predictions_df[f'predicted_{col}']
            combined_df[f'actual_{col}'] = actual_data_df[f'actual_{col}']
        
        combined_df.to_csv('test.csv', index=False)


    def rolling_prediction(self, initial_sequence, full_y):
        """
        Perform rolling prediction based on the initial sequence and full target sequence.

        Args:
        - initial_sequence: The initial sequence to start the prediction from (shape: [50, 18]).
        - full_y: The full target sequence containing all information (shape: [batch_size, 18]).

        Returns:
        - predictions: The predicted future states (shape: [batch_size, 13]).
        """
        sequence = initial_sequence  # Initial sequence of shape [50, 18]
        predictions = []
        batch_size = full_y.size(0)  # Number of steps to predict

        for i in range(batch_size):
            # Predict the next state
            next_state = self(sequence.unsqueeze(0))  # sequence.unsqueeze(0) shape: [1, 50, 18]
            next_state = next_state.squeeze(0)  # Prediction of shape [13]

            # Extract relevant information from full_y for the current step
            full_info = full_y[i]

            # Create a new state of shape [18] to maintain the sequence length
            new_state = torch.zeros(18, device=sequence.device)

            # Assign the predicted values to the appropriate indices in new_state
            new_state[1:14] = next_state  # Indices for ['vt', 'pitch', 'roll', 'hding', 'lat', 'lon', 'alt', 'x', 'y', 'z', 'vx', 'vy', 'vz']

            # Assign the remaining values from full_info
            new_state[0] = full_info[0]  # 'time'
            new_state[14:] = full_info[14:]  # Control inputs: 'elev_stick', 'ailrn_stick', 'ruddr_stick', 'throttle'

            # Append the new state to the sequence and remove the oldest state
            sequence = torch.cat((sequence[1:], new_state.unsqueeze(0)), dim=0)

            # Append the prediction to predictions list
            predictions.append(next_state)

        return torch.stack(predictions)  # Return predictions as a tensor

        
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
    



In [30]:
def seed_everything(seed=42):
    # random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    pl.seed_everything(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
seed_everything()

Global seed set to 42


In [31]:
sequance_length = 20
train_dataset = FlightDataset(train_data, sequance_length)
fitted_scaler = train_dataset.scaler
val_dataset = FlightDataset(val_data, sequance_length, fitted_scaler)
test_dataset = FlightDataset(test_data, sequance_length, fitted_scaler)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

KeyError: "['totl,_time', 'pitch,deg', '_roll,deg', 'lat,deg', 'lon,deg', '_CG,ftMSL', 'X,m', 'Y,m', 'Z,m', '_vX,m/s', '_vY,m/s', '_vZ,_m/s'] not in index"

In [ ]:
# Assuming train_loader is your dataloader
for batch in train_loader:
    # Unpack the batch (assuming it contains inputs and targets)
    inputs, targets = batch
    
    # Print the inputs and targets to understand their structure
    print("Inputs: ", inputs)
    print("Targets: ", targets)
    
    # Print the size of the inputs
    print("Size of inputs: ", inputs.size())
    
    # Break the loop after the first batch
    break


In [ ]:
neptune_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4ZGJmNTQyNy04OWFhLTQ0YzMtYTA5Ni1hNTc1NjJmMGYxMjkifQ==",
    project="kapustya/example-project-tensorflow-keras",
    tags=["test"],
    log_model_checkpoints=False
)

In [ ]:
seed_everything()
torch.set_grad_enabled(True)
input_dim = len(train_data.columns)
hidden_dim = 128
num_layers = 2
output_dim = 13
model = FlightLSTM(input_dim, hidden_dim, num_layers, output_dim)
trainer = pl.Trainer(
    logger = neptune_logger,
    max_epochs=3
    )
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)
model.save_predictions_to_csv(test_loader)
neptune_logger.experiment.stop()
